In [ ]:
import sys
sys.path.append('/home/marcel/pixeltable')

import tensorflow as tf

import pixeltable as pt
from pixeltable.type_system import ImageType
import pixeltable.functions.pil.image as pil
from pixeltable.functions import dict_map
import pixeltable.tf
from pixeltable.functions.tf import TFModelFunction

%load_ext autoreload
%autoreload 2

In [ ]:
cl = pt.Client()
db = cl.get_db('imagenette')
t = db.get_table('data')

In [ ]:
t.count()

In [ ]:
t.show()

# Queries

In [ ]:
t[t.category == 'n03394916'][[t.img, t.img.rotate(90)]].show()

In [ ]:
t[(t.split == 'val') & (t.category >= 'n03445777') & (t.img.height > 100)][
    t.img,
    t.img.rotate(90),
    pil.blend(t.img, t.img.rotate(90), 0.5),
    t.img.height, t.img.width
].show()

### image similarity search

In [ ]:
img_sample = t[t.img].show(1)[0, 0]

In [ ]:
t[t.img.nearest(img_sample)].show()

### keyword search

In [ ]:
t[t.img.matches('door')].show()

# Tensorflow

### convert label to int

In [ ]:
m = t[t.category].categorical_map()
m

In [ ]:
df = t[t.img.convert('RGB').resize((224, 224)), dict_map(t.category, m)]
df.show(1)

### access DataFrame as tf.data.Dataset

In [ ]:
ds = pixeltable.tf.to_dataset(df)
ds

In [ ]:
for elem in ds.take(1):
    print(elem)

### use trained model as UDF

In [ ]:
model = tf.keras.applications.resnet50.ResNet50()
preprocess = tf.keras.applications.resnet50.preprocess_input
model_udf = TFModelFunction(model, ImageType(size=(224, 224), mode=ImageType.Mode.RGB), preprocess=preprocess)

In [ ]:
result = t[model_udf(t.img)].show(1)

In [ ]:
type(result[0, 0])

In [ ]:
result[0, 0].shape